In [26]:
!pip -q install yfinance

In [40]:
import yfinance as yf 
import pandas as pd
import numpy as np
from tqdm import tqdm

# Fetching the required data

In [48]:
avg_spy_r = 0.1141 # Given

In [41]:
def stock_info(ticker:str)->dict:
  """
  Function that returns the beta, 1 year forward dividend and close price for a particular date given the ticker of the stock
  """
  stock = yf.Ticker(ticker)
  stock_i = dict()
  stock_i["beta"] = stock.get_info()["beta"]
  stock_i["div"] = stock.get_info()["dividendRate"]
  stock_i["close"] = stock.history(period="1mo")["Close"].loc["2021-11-05 "]
  return stock_i

In [42]:
tickers = ["AAPL", "PFE", "ED", "XOM", "BAC"]

In [44]:
data = dict()
for t in tqdm(tickers, desc="Fetching data from Yahoo Finance.."):
  data[t] = stock_info(t)

Fetching data from Yahoo Finance: 100%|██████████| 5/5 [00:15<00:00,  3.02s/it]


In [45]:
# Checking our data
print(data)

{'AAPL': {'beta': 1.205714, 'div': 0.88, 'close': 151.27999877929688}, 'PFE': {'beta': 0.665974, 'div': 1.56, 'close': 48.61000061035156}, 'ED': {'beta': 0.190129, 'div': 3.1, 'close': 77.44050598144531}, 'XOM': {'beta': 1.343376, 'div': 3.52, 'close': 64.15776824951172}, 'BAC': {'beta': 1.538701, 'div': 0.84, 'close': 46.970001220703125}}


In [46]:
# In case API call fails due to network issue, we can use the data by manual input
# This might lead to certain changes in decimal values to the consideration of less number of significant digits

# data = {"AAPL":{"beta":1.21, "div":0.88, "close": 151.28}, 
#         "PFE":{"beta":0.67, "div":1.56, "close": 48.61}, 
#         "ED":{"beta":0.19, "div":3.10, "close": 78.21	}, 
#         "XOM":{"beta":1.34, "div":3.52, "close": 65.02},
#         "BAC":{"beta":1.54, "div":0.84, "close": 46.97}
#         }

In [47]:
# Typecasting our data into a dataframe
df = pd.DataFrame.from_dict(data,orient="index")
df

,beta,div,close
AAPL,1.205714,0.88,151.279999
PFE,0.665974,1.56,48.610001
ED,0.190129,3.10,77.440506
XOM,1.343376,3.52,64.157768
BAC,1.538701,0.84,46.970001


# Compute Market Capitalization Rate

Market capitalization rate of a company can be expressed as  

$$r\times \beta$$

where r is the average return of SPY500 over the last 20 years

In [49]:
df["market_cap_r"] = df["beta"]*avg_spy_r 
df

,beta,div,close,market_cap_r
AAPL,1.205714,0.88,151.279999,0.137572
PFE,0.665974,1.56,48.610001,0.075988
ED,0.190129,3.10,77.440506,0.021694
XOM,1.343376,3.52,64.157768,0.153279
BAC,1.538701,0.84,46.970001,0.175566


# Value of Stocks using DDM

$$P = \frac{D}{r}$$

In [54]:
p = df["div"]/ df["market_cap_r"]
p

AAPL      6.396652
PFE      20.529656
ED      142.898505
XOM      22.964629
BAC       4.784531
dtype: float64

# Growth Rate of Dividends using DGM





$$\begin{split} &P = \frac{ D_1 }{ r - g } \\ &\textbf{where:} \\ &P = \text{Current stock price} \\ &g = \text{Constant growth rate expected for} \\ &\text{dividends, in perpetuity} \\ &r = \text{Constant cost of equity capital for the} \\ &\text{company (or rate of return)} \\ &D_1 = \text{Value of next year's dividends} \\ \end{split} $$


$$\therefore g = r - \frac{D_1}{P}$$



In [55]:
g = df["market_cap_r"] - (df["div"]/df["close"])
g*100 # Value in percent

AAPL    13.175494
PFE      4.389547
ED      -1.833701
XOM      9.841445
BAC     15.768203
dtype: float64